<a href="https://colab.research.google.com/github/Tom-Lennox/Kaggle/blob/master/porto_seguro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 【行ったこと】

# LogisticRegression、特に設定しなければrandomと同じくらいのscore.
# auc:0.5がランダムと同一。
# ▼ 

# ▼ 3 4の準備

# ▼ 4 hyper param　の調整


# 【memo】
# future_engneering > paramの調整
# 

In [1]:
# kaggle APIセット
!pip install kaggle

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)
# 「Download 100%.」と表示で成功。

# [kaggle.json]を持参する。
from google.colab import drive
drive.mount('/content/drive')

# jsonファイルを指定の場所に配置
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
# titanicをダウンロードします。
!kaggle competitions download -c porto-seguro-safe-driver-prediction

# jsonファイルを指定の場所に配置
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
# titanicをダウンロードします。
!kaggle competitions download -c porto-seguro-safe-driver-prediction
!ls

# ▼ unzip
!unzip '*.zip'
!ls
# ▲ unzip

# ▼ kaggle notebookの場合
# # !cd ../
# !ls ../input/porto-seguro-safe-driver-prediction

# TRAIN_DATA = '../input/porto-seguro-safe-driver-predicti1on/train.csv'
# df = pd.read_csv(TRAIN_DATA)
# ▲ kaggle notebookの場合

# ▼ 行表示
# ctrl + m + l
# ▲ 行表示

Download 100%.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
cp: cannot create regular file '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, s

In [2]:
import pandas as pd
import numpy as np

from logging import getLogger

TRAIN_DATA = './train.csv'
TEST_DATA =  './test.csv'

logger = getLogger(__name__)

def read_csv(path):
  logger.debug('enter')
  logger.debug('exit')
  return df

def load_train_data():
  logger.debug('enter')
  df = pd.read_csv(TRAIN_DATA)
  logger.debug('exit')
  return df

def load_test_data():
  logger.debug('enter')
  df = pd.read_csv(TEST_DATA)
  logger.debug('exit')
  return df

if __name__ == '__main__':
  print(load_train_data().head())
  print(load_test_data().head())
  # print(load_train_data().info())
  # print(load_test_data().info())

   id  target  ps_ind_01  ...  ps_calc_18_bin  ps_calc_19_bin  ps_calc_20_bin
0   7       0          2  ...               0               0               1
1   9       0          1  ...               0               1               0
2  13       0          5  ...               0               1               0
3  16       0          0  ...               0               0               0
4  17       0          0  ...               1               1               0

[5 rows x 59 columns]
   id  ps_ind_01  ps_ind_02_cat  ...  ps_calc_18_bin  ps_calc_19_bin  ps_calc_20_bin
0   0          0              1  ...               0               0               1
1   1          4              2  ...               1               0               1
2   2          5              1  ...               0               0               0
3   3          0              1  ...               0               0               0
4   4          5              1  ...               0               0               1

In [3]:
# ▼ 1_簡単なsubmit
from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger
from sklearn.linear_model import LogisticRegression
!mkdir result_tmp
# from load_data import load_train_data

DIR = 'result_tmp/'
SAMPLE_SUBMIT_FILE = './sample_submission.csv'

# 結果出力先

if __name__ == '__main__':
  log_fmt = Formatter('%(asctime)s %(name)s %(lineno)d [%(levelname)s][%(funcName)s] %(message)s ')
  handler = StreamHandler()
  handler.setLevel('INFO')
  handler.setFormatter(log_fmt)
  logger.addHandler(handler)
  # logの出力は任意、StreamHandler()を流す
  # 標準出力：INFO　Level 以上
  # ファイル出力：debug level 以上

  handler = FileHandler(DIR + 'train.py.log', 'a')
  handler.setLevel(DEBUG)
  handler.setFormatter(log_fmt)
  logger.setLevel(DEBUG)
  logger.addHandler(handler)

  logger.info('start')
  #logを取るメリット：セクション間の実行時間がわかる。」
  df = load_train_data()

  x_train = df.drop('target', axis=1)
  # targetはdataに入ると過学習になる。
  y_train = df['target'].values
  # valuesにするとpdからnp配列になる。
  use_cols = x_train.columns.values
  # 並び順固定
  logger.debug('train columns: {} {}'.format(use_cols.shape, use_cols))
  logger.info('data preparation end {}'.format(x_train.shape))

  clf = LogisticRegression(random_state=0)
  clf.fit(x_train, y_train)

  logger.info('train end')

  df = load_test_data()

  x_test = df[use_cols].sort_values('id')
  # + 念の為sort
  logger.info('test data load end {}'.format(x_test.shape))
  pred_test = clf.predict_proba(x_test)

  df_submit = pd.read_csv(SAMPLE_SUBMIT_FILE).sort_values('id')
  df_submit['target'] = pred_test

  df_submit.to_csv(DIR + 'submit.csv')
# ▲ 1_簡単なsubmit

2020-05-17 22:43:07,612 __main__ 27 [INFO][<module>] start 
2020-05-17 22:43:09,934 __main__ 38 [INFO][<module>] data preparation end (595212, 58) 
2020-05-17 22:43:16,626 __main__ 43 [INFO][<module>] train end 
2020-05-17 22:43:20,250 __main__ 49 [INFO][<module>] test data load end (892816, 58) 


In [3]:
# ▼ 1_簡単なsubmit
from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger
from sklearn.linear_model import LogisticRegression
  # ▼ 2_cross_validation追加
from sklearn.model_selection import StratifiedKFold
# 各cvの設定で01の引き率を一定にできる。※バスケット分析等は例外。
from sklearn.metrics import log_loss, roc_auc_score
  # ▲ 2_cross_validation追加
!mkdir result_tmp
# from load_data import load_train_data

DIR = 'result_tmp/'
SAMPLE_SUBMIT_FILE = './sample_submission.csv'

# 結果出力先

if __name__ == '__main__':
  log_fmt = Formatter('%(asctime)s %(name)s %(lineno)d [%(levelname)s][%(funcName)s] %(message)s ')
  handler = StreamHandler()
  handler.setLevel('INFO')
  handler.setFormatter(log_fmt)
  logger.addHandler(handler)
  # logの出力は任意、StreamHandler()を流す
  # 標準出力：INFO　Level 以上
  # ファイル出力：debug level 以上

  handler = FileHandler(DIR + 'train.py.log', 'a')
  handler.setLevel(DEBUG)
  handler.setFormatter(log_fmt)
  logger.setLevel(DEBUG)
  logger.addHandler(handler)

  logger.info('start')
  #logを取るメリット：セクション間の実行時間がわかる。」
  df = load_train_data()

  x_train = df.drop('target', axis=1)
  # targetはdataに入ると過学習になる。
  y_train = df['target'].values
  # valuesにするとpdからnp配列になる。
  use_cols = x_train.columns.values
  # 並び順固定
  logger.debug('train columns: {} {}'.format(use_cols.shape, use_cols))
  logger.info('data preparation end {}'.format(x_train.shape))

  # ▼ 2_cross_validation追加
  cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
  # StratifiedKFold：行のindexが返ってくる。

    # ▼ 3
  list_auc_score = []
  list_logloss_score = []
    # ▲ 3
  for train_idx, valid_idx in cv.split(x_train, y_train):
    # 01の比率はtaegetの方にしか寄らないので、xは何でも良い。
    trn_x = x_train.iloc[train_idx, :]
    val_x = x_train.iloc[valid_idx, :]
    # pandasなので.iloc
    # 列も(全部)指定。

    # trn_y = x_train[train_idx]
    # val_y = x_train[valid_idx]
    # miss
    trn_y = y_train[train_idx]
    val_y = y_train[valid_idx]
    # train ⇒ validation　計って評価する、を全部のsplitで行う。
    # 手元cv_scoreとreader_boadのcvが一致するように適正にcloss_validationを切る。
    # 一致しない場合はどこかに偏りが生じている。
    clf = LogisticRegression(random_state=0)
    clf.fit(x_train, y_train)

    pred = clf.predict_proba(val_x)[:, 1]
    sc_logloss = log_loss(val_y, pred)
    sc_auc = roc_auc_score(val_y, pred)

    # ▼ 3
    list_logloss_score.append(sc_logloss)
    list_auc_score.append(sc_auc)
    logger.debug('  logloss: {}, auc: {}'.format(sc_logloss, sc_auc))
    # log見てわかるようにindent下げる。
    # ▲ 3
    # ▼ 3
    # logger.info('logloss: {}, auc: {}'.format(sc_logloss, sc_auc))
    logger.info('logloss: {}, auc: {}'.format(np.mean(sc_logloss), np.mean(list_auc_score)))
    # infoとdebug、何が違うのか？
    # ▲ 3
    # logloss
    # auc
  # ▲ 2_cross_validation追加

  clf = LogisticRegression(random_state=0)
  clf.fit(trn_x, trn_y)

  logger.info('train end')

  df = load_test_data()

  x_test = df[use_cols].sort_values('id')
  # + 念の為sort
  logger.info('test data load end {}'.format(x_test.shape))
  # ▼ 2_cross_validation追加
  pred_test = clf.predict_proba(x_test)
  # pred_test = clf.predict_proba(x_test)[:, 1]
  # ▲ 2_cross_validation追加
  df_submit = pd.read_csv(SAMPLE_SUBMIT_FILE).sort_values('id')
  df_submit['target'] = pred_test

  df_submit.to_csv(DIR + 'submit.csv')
# ▲ 1_簡単なsubmit
# # ▼ log
# !ls ./result_tmp
# !cat ./result_tmp/train.py.log
# # ▲ log

2020-05-21 23:28:05,081 __main__ 33 [INFO][<module>] start 
2020-05-21 23:28:07,348 __main__ 44 [INFO][<module>] data preparation end (595212, 58) 
2020-05-21 23:28:14,645 __main__ 84 [INFO][<module>] logloss: 0.18807471172878062, auc: 0.494807348616394 
2020-05-21 23:28:24,840 __main__ 84 [INFO][<module>] logloss: 0.18887723056374864, auc: 0.49153889099957304 
2020-05-21 23:28:31,895 __main__ 84 [INFO][<module>] logloss: 0.18813862137608495, auc: 0.49210187713531856 
2020-05-21 23:28:38,991 __main__ 84 [INFO][<module>] logloss: 0.18940144559010477, auc: 0.49057707640074 
2020-05-21 23:28:46,018 __main__ 84 [INFO][<module>] logloss: 0.18760810234702324, auc: 0.49191273180231454 
2020-05-21 23:28:51,441 __main__ 94 [INFO][<module>] train end 
2020-05-21 23:28:55,048 __main__ 100 [INFO][<module>] test data load end (892816, 58) 


In [5]:
!ls ./result_tmp
!cat ./result_tmp/train.py.log

train.py.log
2020-05-18 23:57:06,961 __main__ 32 [INFO][<module>] start 
2020-05-18 23:57:06,962 __main__ 17 [DEBUG][load_train_data] enter 
2020-05-18 23:57:09,214 __main__ 19 [DEBUG][load_train_data] exit 
2020-05-18 23:57:09,277 __main__ 42 [DEBUG][<module>] train columns: (58,) ['id' 'ps_ind_01' 'ps_ind_02_cat' 'ps_ind_03' 'ps_ind_04_cat'
 'ps_ind_05_cat' 'ps_ind_06_bin' 'ps_ind_07_bin' 'ps_ind_08_bin'
 'ps_ind_09_bin' 'ps_ind_10_bin' 'ps_ind_11_bin' 'ps_ind_12_bin'
 'ps_ind_13_bin' 'ps_ind_14' 'ps_ind_15' 'ps_ind_16_bin' 'ps_ind_17_bin'
 'ps_ind_18_bin' 'ps_reg_01' 'ps_reg_02' 'ps_reg_03' 'ps_car_01_cat'
 'ps_car_02_cat' 'ps_car_03_cat' 'ps_car_04_cat' 'ps_car_05_cat'
 'ps_car_06_cat' 'ps_car_07_cat' 'ps_car_08_cat' 'ps_car_09_cat'
 'ps_car_10_cat' 'ps_car_11_cat' 'ps_car_11' 'ps_car_12' 'ps_car_13'
 'ps_car_14' 'ps_car_15' 'ps_calc_01' 'ps_calc_02' 'ps_calc_03'
 'ps_calc_04' 'ps_calc_05' 'ps_calc_06' 'ps_calc_07' 'ps_calc_08'
 'ps_calc_09' 'ps_calc_10' 'ps_calc_11' 'ps_calc_12' '